<div style="font-size: 200%; font-weight: bold; color: maroon;">Contador de palabras en SparkR</div>
<!-- v. 1.1 - Paulo Villegas, 2017 -->

Implementamos el típico contador de palabras en un documento de texto, usando SparkR.

Dado que el único API disponible es el de DataFrames, tenemos que hacer todo el proceso usando SparkDataFrames.

# Inicialización

Carga de la librería y creadión de la `SparkSession`

In [ ]:
library(SparkR)
spark <- sparkR.session( "local[*]" );

# Lectura de datos

Leemos un fichero de texto para crear un `SparkDataFrame`. Spark lee línea a línea y crea un DataFrame que contiene una fila para cada línea, contenida en la columna `value` 

In [ ]:
dq <- read.df( '../DATA/DonQuijote.txt.bz2', 'text')

In [ ]:
count(dq)

In [ ]:
printSchema(dq)

# Limpieza

In [ ]:
dqf = filter( dq, dq$value != '' )

In [ ]:
count(dqf)

In [ ]:
head(dqf)

# Separación en palabras

Para separar cada línea en palabras vamos a usar funciones de SPARQ SQL.
La función de SparkR [`selectExpr`](https://spark.apache.org/docs/latest/api/R/selectExpr.html) permite aplicar una 
expresión SQL a un dataframe de Spark. En este caso la expresión contendrá la función `split` aplicada a a la columna que contiene las líneas leídas. El resultado será un vector en cada fila

In [ ]:
linevector = selectExpr(dqf, "split(value, \"[\t ]+\") as colv") 

In [ ]:
head(linevector)

A continuación aplicamos la función [`explode`](https://spark.apache.org/docs/latest/api/R/explode.html) de SparkR a esa columna, generando una fila por cada elemento del vector. Lo llamamos envuelto en [`alias`](https://spark.apache.org/docs/latest/api/R/alias.html) para dar nombre a la nueva columna

In [ ]:
words = select(linevector, alias(explode(linevector$colv), "item"))

In [ ]:
head(words)

In [ ]:
count(words)

# Agrupación + cálculo

Tenemos ahora un SparkDataFrame que contiene palabras (en la columna `value`). Vamos a agrupar por esa columna; el efecto es crear un grupo por cada palabra distinta

In [ ]:
words.grouped = groupBy( words, words$item )

Y ahora usamos una función de agregación, que simplemente cuente el número de filas en cada grupo

In [ ]:
word.counts = summarize( words.grouped, count=n(words$item) )

In [ ]:
head(word.counts)

In [ ]:
# también se podría hacer directamente
head(count(groupBy( words, words$item )))

# Ordenación

Por último, ordenamos en orden decreciente de frecuencia de palabra

In [ ]:
sorted.counts = arrange( word.counts, desc(word.counts$count) )

In [ ]:
head(sorted.counts)